In [1]:
class Imagen:
    def __init__(self, tamaño, color):
        self._tamaño = tamaño
        self._color = color

    @property
    def tamaño(self):
        return self._tamaño

    @tamaño.setter
    def tamaño(self, nuevo_tamaño):
        self._tamaño = nuevo_tamaño

    @property
    def color(self):
        return self._color

    @color.setter
    def color(self, nuevo_color):
        # Verificar si el formato del color es válido
        if isinstance(nuevo_color, tuple) and len(nuevo_color) == 3:
            if all(isinstance(componente, int) and 0 <= componente <= 255 for componente in nuevo_color):
                self._color = nuevo_color
            else:
                raise ValueError("Los componentes de color deben estar entre 0 y 255")
        else:
            raise ValueError("El formato del color debe ser una tupla (r, g, b) con valores entre 0 y 255")

# Ejemplo de uso:
# imagen1 = Imagen(tamaño=(800, 600), color=(0, 0, 255))
# print("Tamaño de la imagen:", imagen1.tamaño)
# print("Color de la imagen:", imagen1.color)

# # Cambiar el tamaño de la imagen
# imagen1.tamaño = (1024, 768)
# print("Nuevo tamaño de la imagen:", imagen1.tamaño)

# # Cambiar el color de la imagen
# imagen1.color = (255, 0, 0)
# print("Nuevo color de la imagen:", imagen1.color)


In [2]:
import numpy as np
import cv2
import tkinter as tk
from PIL import Image, ImageTk
import random


In [3]:
# Función para crear una imagen del gato con un color dado
def crear_imagen_gato(color, size):
    height, width = size

    image = np.ones((height, width, 3), dtype=np.uint8) * 255  # Fondo blanco

    # Coordenadas relativas de las características del gato
    relative_head_center = (0.5, 0.4)
    relative_head_axes = (0.25, 0.17)
    relative_eye_left = (0.43, 0.45)
    relative_eye_right = (0.75, 0.45)
    relative_nose_center = (0.5, 0.55)
    relative_nose_axes = (0.05, 0.03)
    relative_ear_left = [(0.4, 0.3), (0.5, 0.2), (0.6, 0.3)]
    relative_ear_right = [(0.8, 0.3), (0.7, 0.2), (0.6, 0.3)]

    # Convertir las coordenadas relativas a coordenadas absolutas
    head_center = (int(width * relative_head_center[0]), int(height * relative_head_center[1]))
    head_axes = (int(width * relative_head_axes[0]), int(height * relative_head_axes[1]))
    eye_left = (int(width * relative_eye_left[0]), int(height * relative_eye_left[1]))
    eye_right = (int(width * relative_eye_right[0]), int(height * relative_eye_right[1]))
    nose_center = (int(width * relative_nose_center[0]), int(height * relative_nose_center[1]))
    nose_axes = (int(width * relative_nose_axes[0]), int(height * relative_nose_axes[1]))
    ear_left = np.array([[int(width * p[0]), int(height * p[1])] for p in relative_ear_left])
    ear_right = np.array([[int(width * p[0]), int(height * p[1])] for p in relative_ear_right])

    # Dibujar la cabeza del gato
    cv2.ellipse(image, head_center, head_axes, 0, 0, 360, color, -1)

    # Dibujar los ojos
    cv2.circle(image, eye_left, int(0.05 * height), (0, 0, 0), -1)
    cv2.circle(image, eye_right, int(0.05 * height), (0, 0, 0), -1)

    # Dibujar la nariz
    cv2.ellipse(image, nose_center, nose_axes, 0, 0, 360, (0, 0, 0), -1)

    # Dibujar las orejas
    cv2.fillPoly(image, [ear_left], color)
    cv2.fillPoly(image, [ear_right], color)

    return image

In [4]:
import math

# Función para calcular la distancia euclidiana entre dos colores en el espacio RGB
def euclidean_distance(color1, color2):
    return math.sqrt(sum((c1 - c2) ** 2 for c1, c2 in zip(color1, color2)))

In [5]:
def calculate_fitness(image_size, target_size):
    # Calcula la diferencia absoluta entre el tamaño actual y el tamaño deseado
    diff_width = abs(image_size[0] - target_size[0])
    diff_height = abs(image_size[1] - target_size[1])

    # Cuanto menor sea la diferencia, mejor es el ajuste, por lo tanto,
    # el fitness será la inversa de la suma de las diferencias
    fitness = 1 / (diff_width + diff_height + 1)  # +1 para evitar divisiones por cero

    return fitness

In [6]:
import random

# Función para generar un color aleatorio en el rango RGB
def generate_random_color():
    return [random.randint(0, 255) for _ in range(3)]

In [7]:
def generate_random_size():
    return [random.randint(50, 150) for _ in range(2)]

In [8]:
# Función para generar una población inicial de colores aleatorios
def generate_initial_population(population_size):
    return [generate_random_color() for _ in range(population_size)]

In [9]:
# Función para generar una población inicial de colores aleatorios
def generate_size_initial_population(population_size):
    return [generate_random_size() for _ in range(population_size)]

In [10]:
# Función de selección de torneo
def tournament_selection(population, target_color,k=3):
    tournament_contestants = random.sample(population, k)
    return min(tournament_contestants, key=lambda x: euclidean_distance(x,target_color))

In [11]:
def tournament_selection_Size(population, target_size, k=3):
    # Selecciona aleatoriamente k individuos de la población para el torneo
    tournament_contestants = random.sample(population, k)
    
    # Calcula el fitness de cada individuo en el torneo
    fitness_scores = [calculate_fitness(individual, target_size) for individual in tournament_contestants]
    
    # Encuentra al ganador del torneo (el individuo con el mejor fitness)
    winner_index = fitness_scores.index(max(fitness_scores))
    winner = tournament_contestants[winner_index]
    
    return winner


In [12]:
# Función de cruce (crossover) de un punto
def single_point_crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1) - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child

In [13]:
# Función de mutación
def mutation(individual, mutation_rate=0.8):
    mutated_individual = individual[:]
    if random.random() < mutation_rate:
        mutation_index = random.randint(0, len(individual) - 1)
        mutated_individual[mutation_index] = random.randint(0, 255)
    return mutated_individual

# Ejemplo de mutación
# individual = [200, 100, 51]
# mutated_individual = mutation(individual)
# print("Individuo mutado:", mutated_individual)

In [14]:
# Función para evolucionar la población durante una generación de manera generacional
def evolve_population(population,target_color, mutation_rate=0.8, crossover_rate=0.6):
    new_population = []

    # Cantidad de individuos que se seleccionarán para el cruce y la mutación
    crossover_count = int(len(population) * crossover_rate)
    mutation_count = len(population) - crossover_count

    # Realizar cruce
    for _ in range(crossover_count):
        parent1 = tournament_selection(population,target_color)
        parent2 = tournament_selection(population,target_color)
        child = single_point_crossover(parent1, parent2)
        new_population.append(child)

    # Realizar mutación
    for _ in range(mutation_count):
        individual = tournament_selection(population,target_color)
        mutated_individual = mutation(individual, mutation_rate)
        new_population.append(mutated_individual)

    return new_population

In [15]:
# Función para determinar si se ha alcanzado el criterio de parada
def has_converged(population, target_color, threshold=1):
    return all(euclidean_distance(individual, target_color) < threshold for individual in population)

In [16]:
# Crear la ventana de la interfaz
root = tk.Tk()
root.title("Gatos!")

# Crear un lienzo para colocar las imágenes de los gatos
canvas = tk.Canvas(root, width=1000, height=1200)
canvas.pack()

# Lista para almacenar las imágenes y evitar que Python las elimine
imagenes = []

# Lista para almacenar los objetos PhotoImage de los gatos
cat_photos = []

# Coordenadas iniciales para la primera imagen de gato
x, y = 110, 50


target_color = [255, 0, 0]
population_size = 10
mutation_rate = 0.8
max_generations = 10000

population = generate_initial_population(population_size)
size_population = generate_size_initial_population(population_size)
print(size_population)
for i, (color, size) in enumerate(zip(population, size_population)):
    imagen = Imagen(tamaño=size, color=color)
    image = crear_imagen_gato(imagen.color, imagen.tamaño)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    photo = ImageTk.PhotoImage(image)

    cat_photo = canvas.create_image(x, y, image=photo)
    cat_photos.append(photo)

    x += 300  
    if x >= 800:
        x = 300
        y += 150


# Color objetivo al que queremos que evolucionen los gatos
target_color = [255,0, 0 ]

# Evolucionar los colores de los gatos hacia el color objetivo en tiempo real
# evolve_cat_colors(canvas, target_color,population)
# Mostrar ventana
root.mainloop()

[[96, 73], [121, 119], [70, 65], [149, 52], [116, 103], [114, 93], [70, 117], [89, 82], [107, 86], [121, 105]]


In [17]:
# Crear cinco objetos de la clase Imagen
imagenes_gatos = [
    Imagen(tamaño=(random.randint(100,150), random.randint(100,190)), color=(255, 0, 0)),   # Rojo
    Imagen(tamaño=(random.randint(100,150), random.randint(100,190)), color=(0, 255, 0)),   # Verde
    Imagen(tamaño=(random.randint(100,150), random.randint(100,190)), color=(0, 0, 255)),   # Azul
    Imagen(tamaño=(random.randint(100,150), random.randint(100,190)), color=(255, 255, 0)), # Amarillo
    Imagen(tamaño=(random.randint(100,150), random.randint(100,190)), color=(255, 0, 255))  # Magenta
]

# Dibujar los gatos
for i, img_gato in enumerate(imagenes_gatos, start=1):
    img = crear_imagen_gato(img_gato.color, img_gato.tamaño)
    cv2.imshow(f"Gato {i}", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()